In [60]:
import sys, os
sys.path.append('/elo_bench')
import plotly.express as px

import gradio as gr
from pathlib import Path
import pandas as pd

from data import ARENA_K
from elo_rating.rating_helper import get_elo_results_from_battles_data, get_bootstrap_medium_elo, get_bootstrap_result

from datamodel.elo_rating_history import EloRatingHistory, BattleOutcomes
import numpy as np
from collections import defaultdict
from elo_rating.rating_evaluator import evaluate_rank_consistency, evaluate_winrate_at_historypoint
from elo_rating import RatingEntity

from tqdm import tqdm

pairwise model battles more -> winrate N:N more stable, rank N:1 more consistenct

Input is battled pairs with columns(modela, modelb and winnwe) that's all

carings:
    - 1:1 winrate at each battle round of the pair
    - [P2]N:M ranking on natual battle

In [61]:
result_dir = r'/elo_bench/results/google_quora_alpaca_10629_test3'
battled_outcome_file = Path(result_dir)/'battled_pairs.csv'
FIRST_N_BATTLES = None
battled_outcome_df = pd.read_csv(battled_outcome_file, nrows=FIRST_N_BATTLES)
# valid_winner = ['model_a', 'model_b', 'tie', 'tie(all_bad)']
valid_winner = ['model_a', 'model_b']
battled_outcome_df = battled_outcome_df[battled_outcome_df['winner'].isin(valid_winner)]


In [62]:
battled_outcome_df

,Unnamed: 0,model_a,model_b,winner
0,0,lmsys/vicuna-13b-v1.5,lmsys/vicuna-7b-v1.5,model_a
1,1,lmsys/vicuna-13b-v1.5,lmsys/vicuna-7b-v1.5,model_b
2,2,lmsys/vicuna-13b-v1.5,lmsys/vicuna-7b-v1.5,model_a
3,3,lmsys/vicuna-13b-v1.5,lmsys/vicuna-7b-v1.5,model_b
4,4,lmsys/vicuna-13b-v1.5,lmsys/vicuna-7b-v1.5,model_a
...,...,...,...,...
11849,11849,WizardLM/WizardLM-7B-V1.0,chavinlo/alpaca-13b,model_b
11860,11860,WizardLM/WizardLM-7B-V1.0,chavinlo/alpaca-13b,model_a
11892,11892,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_b
11992,11992,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_a


In [63]:
models = sorted(battled_outcome_df['model_a'].unique())
models

['WizardLM/WizardLM-7B-V1.0',
 'Xwin-LM/Xwin-LM-13B-V0.1',
 'Xwin-LM/Xwin-LM-7B-V0.1',
 'chavinlo/alpaca-13b',
 'chavinlo/alpaca-native',
 'gpt-35-turbo',
 'gpt-4-turbo',
 'lmsys/vicuna-13b-v1.5',
 'lmsys/vicuna-33b-v1.3',
 'lmsys/vicuna-7b-v1.5',
 'meta-llama/Llama-2-13b-chat-hf',
 'meta-llama/Llama-2-70b-chat-hf',
 'meta-llama/Llama-2-7b-chat-hf']

In [64]:
# swap model_a and model_b if model_a is not the first model
def swap_model_a_b(row, models_in_order):
    if models_in_order.index(row['model_a']) > models_in_order.index(row['model_b']):
        return row
    else:
        return pd.Series([row['model_b'], row['model_a'], row['winner']], index=['model_a', 'model_b', 'winner'])
    
# Apply the function to each row
battled_outcome_df = battled_outcome_df.apply(swap_model_a_b, args=(models,), axis=1)

In [65]:
battled_outcome_df

,Unnamed: 0,model_a,model_b,winner
0,NaN,lmsys/vicuna-7b-v1.5,lmsys/vicuna-13b-v1.5,model_a
1,NaN,lmsys/vicuna-7b-v1.5,lmsys/vicuna-13b-v1.5,model_b
2,NaN,lmsys/vicuna-7b-v1.5,lmsys/vicuna-13b-v1.5,model_a
3,NaN,lmsys/vicuna-7b-v1.5,lmsys/vicuna-13b-v1.5,model_b
4,NaN,lmsys/vicuna-7b-v1.5,lmsys/vicuna-13b-v1.5,model_a
...,...,...,...,...
11849,NaN,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_b
11860,NaN,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_a
11892,11892.0,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_b
11992,11992.0,chavinlo/alpaca-13b,WizardLM/WizardLM-7B-V1.0,model_a


In [66]:
group_by_ab = battled_outcome_df.groupby(['model_a', 'model_b'])
group_by_ab

In [67]:
# Create a dictionary to store each group as a DataFrame
group_dfs = {}

for (model_a, model_b), group_df in group_by_ab:
    # Create a DataFrame for each group
    group_dfs[(model_a, model_b)] = group_df.reset_index(drop=True)

# Accessing a specific group, for example ('A', 'X')
specific_group_df = group_dfs.get(('gpt-35-turbo', 'Xwin-LM/Xwin-LM-7B-V0.1'), "Group not found")
print(specific_group_df)

     Unnamed: 0       model_a                  model_b   winner
0           NaN  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b
1         732.0  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_a
2         733.0  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b
3         734.0  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b
4           NaN  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b
..          ...           ...                      ...      ...
115         NaN  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_a
116         NaN  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_a
117         NaN  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_a
118     10847.0  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b
119     10915.0  gpt-35-turbo  Xwin-LM/Xwin-LM-7B-V0.1  model_b

[120 rows x 4 columns]


In [68]:
from elo_rating.rating_evaluator import compute_actual_winrate_awinb
model_a = 'lmsys/vicuna-7b-v1.5'
model_b = 'lmsys/vicuna-13b-v1.5'
specific_group_df = group_dfs.get((model_a, model_b), "Group not found")
print(specific_group_df)

actual_winrate_history = []
for i in range(1, len(specific_group_df)):
    actual_winrate_awinb = compute_actual_winrate_awinb(specific_group_df.head(i), model_a, model_b)
    # print(f'{i} {actual_winrate_awinb}')
    
    # record the actual winrate
    actual_winrate_history.append({
        'actual_winrate': actual_winrate_awinb,
        'num_battles': i
    })
    
actual_winrate_history_df = pd.DataFrame(actual_winrate_history)
actual_winrate_history_df

     Unnamed: 0               model_a                model_b   winner
0           NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a
1           NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_b
2           NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a
3           NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_b
4           NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a
..          ...                   ...                    ...      ...
106     11076.0  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_b
107         NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a
108         NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_b
109         NaN  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a
110     11090.0  lmsys/vicuna-7b-v1.5  lmsys/vicuna-13b-v1.5  model_a

[111 rows x 4 columns]


,actual_winrate,num_battles
0,1.000000,1
1,0.500000,2
2,0.666667,3
3,0.500000,4
4,0.600000,5
...,...,...
105,0.603774,106
106,0.598131,107
107,0.601852,108
108,0.596330,109


In [69]:
# plot the actual winrate and the delta of actual winrate
fig = px.line(actual_winrate_history_df, x="num_battles", y="actual_winrate", title='Actual winrate')

In [70]:
fig

In [71]:
# calculate the delta of actual winrate
actual_winrate_history_df['delta_actual_winrate'] = actual_winrate_history_df['actual_winrate'].diff().abs()

In [72]:
actual_winrate_history_df

,actual_winrate,num_battles,delta_actual_winrate
0,1.000000,1,NaN
1,0.500000,2,0.500000
2,0.666667,3,0.166667
3,0.500000,4,0.166667
4,0.600000,5,0.100000
...,...,...,...
105,0.603774,106,0.003774
106,0.598131,107,0.005643
107,0.601852,108,0.003721
108,0.596330,109,0.005522


In [73]:
# plot the delta of actual winrate
fig = px.line(actual_winrate_history_df, x="num_battles", y="delta_actual_winrate", title='Delta of actual winrate')
fig

In [74]:
# plot the delta of actual winrate and the actual winrate together
fig = px.line(actual_winrate_history_df, x="num_battles", y="delta_actual_winrate", title='Delta of actual winrate')
fig.add_scatter(x=actual_winrate_history_df['num_battles'], y=actual_winrate_history_df['actual_winrate'], mode='lines', name='actual_winrate')
fig

In [76]:
from elo_rating.rating_evaluator import compute_actual_winrate_awinb
# iterate all the groups
delta_actual_winrate_history = []
for (model_a, model_b), group_df in group_dfs.items():
    actual_winrate_history = []
    for i in range(1, len(group_df)):
        actual_winrate_awinb = compute_actual_winrate_awinb(group_df.head(i), model_a, model_b)
        # print(f'{i} {actual_winrate_awinb}')

        # record the actual winrate
        actual_winrate_history.append({
            'actual_winrate': actual_winrate_awinb,
            'num_battles': i,
            'models': f'{model_a} vs {model_b}'
        })

    actual_winrate_history_df = pd.DataFrame(actual_winrate_history)
    actual_winrate_history_df['delta_actual_winrate'] = actual_winrate_history_df['actual_winrate'].diff().abs()
    actual_winrate_history_df['model_a'] = model_a
    actual_winrate_history_df['model_b'] = model_b
    delta_actual_winrate_history.append(actual_winrate_history_df)

In [77]:
delta_actual_winrate_history

[    actual_winrate  num_battles  \
 0         1.000000            1   
 1         1.000000            2   
 2         0.666667            3   
 3         0.750000            4   
 4         0.800000            5   
 ..             ...          ...   
 95        0.375000           96   
 96        0.371134           97   
 97        0.367347           98   
 98        0.363636           99   
 99        0.360000          100   
 
                                                models  delta_actual_winrate  \
 0   Xwin-LM/Xwin-LM-13B-V0.1 vs WizardLM/WizardLM-...                   NaN   
 1   Xwin-LM/Xwin-LM-13B-V0.1 vs WizardLM/WizardLM-...              0.000000   
 2   Xwin-LM/Xwin-LM-13B-V0.1 vs WizardLM/WizardLM-...              0.333333   
 3   Xwin-LM/Xwin-LM-13B-V0.1 vs WizardLM/WizardLM-...              0.083333   
 4   Xwin-LM/Xwin-LM-13B-V0.1 vs WizardLM/WizardLM-...              0.050000   
 ..                                                ...                   ...   
 95  X

In [78]:
# plot the delta of actual winrate and the actual winrate as two subplots in vertial direction
# Define the layout of subplots (e.g., 3 columns)
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
num_columns = 1
num_rows = math.ceil(len(group_by_ab) / num_columns)

# Create subplots
fig = make_subplots(rows=num_rows, cols=num_columns, subplot_titles=[his['models'][0] for his in delta_actual_winrate_history])
# Populate each subplot
for i, his in enumerate(delta_actual_winrate_history, start=1):
    row = math.ceil(i / num_columns)
    col = i - (row - 1) * num_columns
    print(f'row={row}, col={col}')
    fig.add_scatter(x=his['num_battles'], y=his['actual_winrate'], mode='lines', name=his['models'][0], row=row, col=col)
    # add y legend name
    fig.update_yaxes(title_text="actual_winrate", row=row, col=col)
    # add x legend name
    fig.update_xaxes(title_text="num_battles", row=row, col=col)
    
# Update layout
fig.update_layout(height=300*num_rows, width=900*num_columns, title_text="Actual winrate")
# hide legend
fig.update_layout(showlegend=False)
fig

row=1, col=1
row=2, col=1
row=3, col=1
row=4, col=1
row=5, col=1
row=6, col=1
row=7, col=1
row=8, col=1
row=9, col=1
row=10, col=1
row=11, col=1
row=12, col=1
row=13, col=1
row=14, col=1
row=15, col=1
row=16, col=1
row=17, col=1
row=18, col=1
row=19, col=1
row=20, col=1
row=21, col=1
row=22, col=1
row=23, col=1
row=24, col=1
row=25, col=1
row=26, col=1
row=27, col=1
row=28, col=1
row=29, col=1
row=30, col=1
row=31, col=1
row=32, col=1
row=33, col=1
row=34, col=1
row=35, col=1
row=36, col=1
row=37, col=1
row=38, col=1
row=39, col=1
row=40, col=1
row=41, col=1
row=42, col=1
row=43, col=1
row=44, col=1
row=45, col=1
row=46, col=1
row=47, col=1
row=48, col=1
row=49, col=1
row=50, col=1
row=51, col=1
row=52, col=1
row=53, col=1
row=54, col=1
row=55, col=1
row=56, col=1
row=57, col=1
row=58, col=1
row=59, col=1
row=60, col=1
row=61, col=1
row=62, col=1
row=63, col=1
row=64, col=1
row=65, col=1
row=66, col=1
row=67, col=1
row=68, col=1
row=69, col=1
row=70, col=1
row=71, col=1
row=72, col=1
r

In [95]:
# each group plot one figure instead of subplot
for i, his in enumerate(delta_actual_winrate_history):
    # create a figure
    fig = px.line(his, x="num_battles", y="actual_winrate", title=f'Actual winrate {his["models"][0]}',  marker_color='black')
    # add delta of actual winrate
    fig.add_scatter(x=his['num_battles'], y=his['delta_actual_winrate'], mode='lines', name='delta_actual_winrate')

    # set the y axis range
    fig.update_yaxes(range=[0, 1])
    
    # calcuate moving average or moving standard deviation of actual winrate
    window_size = 20  # Define window size for moving calculation
    fig.add_scatter(x=his['num_battles'], y=his['actual_winrate'].rolling(window_size).mean(), mode='lines', name='moving average')
    # set alpha to 0.5 to make it transparent
    fig.add_scatter(x=his['num_battles'], y=his['actual_winrate'].rolling(window_size).std(), mode='lines', name='moving std')
    
    his['moving_std'] = his['actual_winrate'].rolling(window_size).std()
    stability_threshold = 0.01  # Define your threshold
    his['is_stable'] = his['moving_std'] < stability_threshold
    
    # count of his['is_stable'] = true is bigger than 0
    if len(his[his['is_stable']])>0:
        # the stable point is the last if begining of stable range
        stable_point = his[his['is_stable']].iloc[-1]
        # find the previous point until the prev point is not stable
        while stable_point['is_stable']:
            stable_point = his.iloc[stable_point.name-1]
        
        # stable_point = his[his['is_stable']].iloc[-1]
        print(f"The data becomes stable at time {stable_point['num_battles']}.")
        
        # plot the stable point
        fig.add_scatter(x=[stable_point['num_battles']], y=[stable_point['actual_winrate']], mode='markers', name='stable point')
    else:
        print("The data is not stable yet.")

    # show the figure
    fig.write_image(rf'/elo_bench/plots/actual_winrate_his/actual_winrate_{i}.png')
    
    
# show all the figures
fig.show()

The data becomes stable at time 93.
The data becomes stable at time 85.
The data becomes stable at time 82.


The data is not stable yet.
The data becomes stable at time 86.
The data becomes stable at time 119.
The data becomes stable at time 92.
The data becomes stable at time 77.
The data becomes stable at time 113.
The data becomes stable at time 66.
The data becomes stable at time 83.
The data becomes stable at time 66.
The data becomes stable at time 112.
The data becomes stable at time 97.
The data becomes stable at time 91.
The data becomes stable at time 90.
The data becomes stable at time 104.
The data becomes stable at time 92.
The data becomes stable at time 125.
The data becomes stable at time 89.
The data becomes stable at time 123.
The data becomes stable at time 66.
The data becomes stable at time 77.
The data becomes stable at time 104.
The data becomes stable at time 73.
The data becomes stable at time 98.
The data becomes stable at time 103.
The data becomes stable at time 112.
The data becomes stable at time 106.
The data becomes stable at time 96.
The data becomes stable at

In [ ]:
len(delta_actual_winrate_history)

78